# Capstone Project

# Libraries

In [141]:
# Import Library 

import re
import requests
from re import search

import unicodedata
from bs4 import BeautifulSoup
from bs4 import Tag

from bs4 import NavigableString
from bs4 import SoupStrainer
from pprint import pprint

import numpy as np
from numpy import array
import pandas as pd 

import csv
import pickle

from collections import Counter
import datetime

from IPython.display import display
from IPython.core import display as ICD
pd.set_option('display.max_columns', 100)


# Define Text Normalization Function

In [2]:
def restore_windows_1252_characters(restore_string):
    """
    Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.
    """

def to_windows_1252(match):
    try:
        return bytes([ord(match.group(0))]).decode('windows-1252')
    except UnicodeDecodeError:
        # No character at the corresponding code point: remove it.
        return ''

    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [3]:
def grab_doc_content( brand, CIK ):
    
    company = {}
    
    company['auto'] = {}
    auto = {brand : CIK
           }
    company['auto'] = auto 

    key = list(auto.copy().values())
    
    return key

In [4]:
grab_doc_content('Google', '1652044')

['1652044']

In [5]:
master_filings_dict = {}
    
file_code = {}
    
file_text = {}

Accession_Number_URL = {}

In [6]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in grab_doc_content('Google', '1652044')]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in grab_doc_content('Google', '1652044'):

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)


In [7]:
def url_maker(dir_url):
    dir_url_list = [dir_url.format(x) for x in grab_doc_content('Google', '1652044')]

    #print('Directory URL: {}'.format(dir_url_list))
    doc_url_list = [] 

    # FOR-loop yielding Accession Numbers from CIK/URL Directory.
    for CIK_num in grab_doc_content('Google', '1652044'):

        doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
        doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
        
        doc_url_list.append(doc_url_new)
        
        url_lists = zip( dir_url_list , doc_url_list )
        
    return url_lists

In [9]:
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
for dir_url, doc_url in url_maker(url): 
    print(1, dir_url)
    print(2, doc_url)

1 https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1652044&type=10-&dateb=&owner=include&count=100
2 https://www.sec.gov/Archives/edgar/data/1652044/{xx}/{yy}.txt


In [10]:
Accession_Number_URL = {}
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

for dir_url, doc_url in url_maker(url):
    
    response = requests.get(dir_url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [ cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', dir_url) ][0]
    CIKK = {cikk : accessionnumber}
    
    for (a,b) in accession_numbers: 
        master_filings_dict[b] = {}
        master_filings_dict[b]['sec_header_content'] = {}
        master_filings_dict[b]['filing_documents'] = None

        doc_url_single = doc_url.format(xx = a, yy = b)

        file_url_list = []

        

        file_url_list.append( doc_url_single )
        
        Accession_Number_URL.update({ b : file_url_list})

In [11]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    #if AN == '0001326801-14-000068':
     #   break
        
    #else:
        A_N.append(AN)
        print(url)
print('\n', A_N)
print(len(A_N))

['https://www.sec.gov/Archives/edgar/data/1652044/000165204420000021/0001652044-20-000021.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204420000008/0001652044-20-000008.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204419000032/0001652044-19-000032.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204419000023/0001652044-19-000023.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204419000015/0001652044-19-000015.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000119312519028757/0001193125-19-028757.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204418000035/0001652044-18-000035.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204418000027/0001652044-18-000027.txt']
['https://www.sec.gov/Archives/edgar/data/1652044/000165204418000016/0001652044-18-000016.txt']
['https://www.sec.gov/Archives/edgar/dat

## Save each Filing.

In [ ]:
# Using a list comprehension if statement to handle empty <hr> tags.
example_string = ['<hr/>', '<hr/>', '<hr style="page-break-after:always"/>', '<hr/>', '<hr/>', '<hr style="page-break-after:always"/>']

# But remember 'all_thematic_breaks' is a BS object. 
example_page_num = [ example_break + 'hehe' for example_break in example_string if len(example_break) > 6]
example_page_num

In [ ]:
# TESTING
url = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204420000021/0001652044-20-000021.txt'
url2 = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204420000008/0001652044-20-000008.txt'
# grab the response
response = requests.get(url2)
master_document_dict = {}
# Soupify
# pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
soup = BeautifulSoup(response.content, 'lxml')
filing_document = soup.find('document')

# Parsing
# Document type ->> document_id
document_id = filing_document.type.find(text=True, recursive=False).strip()
# Document sequence ->> document_sequence
document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
# Document filename ->> document_filename
document_filename = filing_document.filename.find(text=True, recursive=False).strip()
# Document description ->> document_description
document_description = filing_document.description.find(text=True, recursive=False).strip()

# Storage    
# initalize our document dictionary
master_document_dict[document_id] = {}

# add the different parts, we parsed up above.
master_document_dict[document_id]['document_sequence'] = document_sequence
master_document_dict[document_id]['document_filename'] = document_filename
master_document_dict[document_id]['document_description'] = document_description

# Scraping
# grab the text portion of the document, this will be used to split the document into pages.
filing_doc_text = filing_document.find('text').extract()

# find all the thematic breaks, these help define page numbers and page breaks.
all_thematic_breaks = filing_doc_text.find_all('hr')
print(all_thematic_breaks)
# Locate and store page number via list comprehension.
all_page_numbers = [thematic_break.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]
print(all_page_numbers)

In [12]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0001326801-14-000068':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()
        
        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('hr')
        
        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]

        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))

--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 65 page(s) found.
There was 65 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001652044-20-000021 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 97 page(s) found.
There was 101 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001652044-20-000008 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 49 page(s) found.
There was 53 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001652044-19-000032 were parsed and stored.


In [ ]:
string_test = master_filings_dict['0001652044-20-000021']['filing_documents']['10-Q']['pages_code']
string_test

## Anchor Check Function



In [15]:
# Variations of HTML anchors.
CBS_A = 'CONDENSED CONSOLIDATED BALANCE SHEETS'
CBS_B = 'CONSOLIDATED BALANCE SHEETS'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


In [40]:
QorK

{'10-Q': ['0001652044-20-000021',
  '0001652044-19-000032',
  '0001652044-19-000023',
  '0001652044-19-000015',
  '0001652044-18-000035',
  '0001652044-18-000027',
  '0001652044-18-000016',
  '0001652044-17-000042',
  '0001652044-17-000026',
  '0001652044-17-000014',
  '0001652044-16-000038',
  '0001652044-16-000032',
  '0001652044-16-000021',
  '0001652044-15-000005'],
 '10-K': ['0001652044-20-000008',
  '0001652044-19-000004',
  '0001652044-18-000007',
  '0001652044-17-000008',
  '0001652044-16-000012']}

In [49]:
# TESTING: CONDENSED CONSOLIDATED BALANCE SHEETS
for k, v in QorK.items():
        
    for i, vv in enumerate(v):
#         print('\n', i, vv)
        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code'] 
        print('\n', k, '          ', vv, '\n')

        for page in pages:
            if anchor_check(page) and ('in millions' in page):
#                 print('\n', 'Pass Anchor check')

                # Convert string to BS object
                soup = BeautifulSoup(page, 'html5')

                # then get all the rows in the table.
                table_rows = soup.find_all('tr')

                if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
                    del table_rows[0]
#                     print('another table row deleted', '\n')
                single_table = []

                # Loop through each column, adding in three structures to single_table.
                for tr_post, tr in enumerate(table_rows):

                    td = tr.find_all('td')

                    column = [tr.text for tr in td]
                    if tr_post <10 and len(column)<10:
                        
                        print( tr_post, len(column), column)



 10-Q            0001652044-20-000021 

0 6 ['', '', '', '', '', '']
1 2 ['Table of Contents', 'Alphabet Inc.']
3 4 ['', 'As of December 31, 2019', '', 'As ofMarch 31, 2020']
4 4 ['', '', '', '(unaudited)']
5 4 ['Assets', '', '', '']
6 4 ['Current assets:', '', '', '']
7 8 ['Cash and cash equivalents', '$', '18,498\xa0', '\xa0', '', '$', '19,644\xa0', '\xa0']
8 6 ['Marketable securities', '101,177\xa0', '\xa0', '', '97,585\xa0', '\xa0']
9 6 ['Total cash, cash equivalents, and marketable securities', '119,675\xa0', '\xa0', '', '117,229\xa0', '\xa0']

 10-Q            0001652044-19-000032 

0 2 ['', '']
1 2 ['Table of Contents', 'Alphabet Inc.']
2 2 ['', '']
3 2 ['PART I.', 'FINANCIAL INFORMATION']
4 2 ['', '']
5 2 ['ITEM 1.', 'FINANCIAL STATEMENTS']
6 1 ['']
7 8 ['', '', '', '', '', '', '', '']
8 4 ['\xa0', 'As of December 31, 2018', '\xa0', 'As ofSeptember 30, 2019']
9 4 ['\xa0', '\xa0', '\xa0', '(unaudited)']

 10-Q            0001652044-19-000023 

0 2 ['', '']
1 2 ['Table of Conten

## Cleaning Columns Function


In [27]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [16]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

### Cleaning Functions.

In [17]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

In [133]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        try:
            ele= element.encode('cp1252').decode("utf-8", "ignore")
                
        except:
            ele= element.encode('latin1').decode("utf-8", "ignore")
            
        if ele =='':
            continue
            
        ele_norm = unicodedata.normalize('NFKD', ele)
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif ele_norm not in ['\n$', '\n\xa0', '\n', '\xa0']:
            
            n_element = ele_norm.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)
            
            result = [int(clean_ele) * m for clean_ele in res]

            column_list.extend(result)

    return column_list
    

## Cleaning Tables Function

In [145]:
# New and imporved.
def Quarter( column , k):
    
#     print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print('new_column used for year', new_column)
    
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())),letter.replace('of',  '').split()[1] )[0]
            
            Q_num = month_dict[month] 
#             print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
#     print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    if k == '10-Q':
    
        Q_final =['Quarter'] + Q[1:]
        
    if k == '10-K':
        Q_final =['Quarter'] + Q[1:]
#         Q_final.insert(1, 0)
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
#     print(f'Y - {Y}')   # Y - [[], [], [['2015']], [['2016']]]

    Y = [item for sublist in Y for item in sublist ][1]
#     print('this is the interger version of Y - ', int(Y[0]))
    
#     if k == '10-K' and int(Y[0]) < 2017:
#         Y = [str(int(Y[0]) + 1)]
#         print(f'Changed Y into {Y}')
        
#     print(f'Y - {Y}')
    
    Y.insert(0, 'Year')
#     Y.insert(1, 0)
    
    Q_final.insert(1,0)
    Y.insert(1,0)
    print( '\n', 'Q_final', Q_final, 'Y', Y)
    return Q_final, Y

In [146]:
def table_extractor( page, multiplier, k , year):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
#     print(f'multiplier - {multiplier}')
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]
#         print( tr_post, len(column), column)
        # 10-Q 
        if k == '10-Q' and year > 19 :
            
            if tr_post == 3:

                # Quarter Column
                col_Q, col_year = Quarter( column, k )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 5:

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        elif k == '10-Q' and 15< year < 20:
            
            if tr_post == 8:

                # Quarter Column
                col_Q, col_year = Quarter( column, k )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )
                
            elif tr_post > 9:

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        elif k == '10-Q' and year< 16:
            
            if tr_post == 6:

                # Quarter Column
                col_Q, col_year = Quarter( column, k )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 8:

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
                
            if tr_post == 4:
                
               # Quarter Column
                col_Q, col_year = Quarter( column, k )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 5: 
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [22]:
def df_clean( single_table ):
    #print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  

    #table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [143]:
def table(page_code, k, year):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if anchor_check(page) and search(r'Million', page, flags=re.I):
            
            single_table = table_extractor( page, 10**6, k, year )
            
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif anchor_check(page) and  search(r'thousand', page, flags=re.I):
            
            single_table = table_extractor( page, 10**3, k, year )
            
            table_df = df_clean(single_table)
            
            doc_tables.append(table_df)
            
                
    return doc_tables
    

# Eploring the Data 

In [24]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [25]:
explore_dict

{'10-Q': ['0001652044-20-000021', '0001652044-15-000005'],
 '10-K': ['0001652044-20-000008', '0001652044-16-000012']}

In [158]:
for QK, N_O in QorK.items():
    
    for an_postition, acc_num in enumerate(N_O): 
        
        year = int(acc_num.split(sep='-')[1])
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        
        test_df = table(pages, QK, year)
        table_df = test_df[0].drop([1]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        
        ICD.display(table_df)


 This is 0001652044-20-000021 a 10-Q document.

 Q_final ['Quarter', 0, 'Q1'] Y ['Year', 0, '2020']
Shape -  (1, 44)


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Income taxes receivable,Inventory,Other current assets,Total current assets,Non marketable investments,Deferred income taxes,Property and Equipment,Operating lease assets,Intangible assets,Goodwill,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,NaN,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2020,None,19644000000,97585000000,117229000000,21825000000,1910000000,889000000,5165000000,147018000000,12367000000,730000000,76747000000,11219000000,1840000000,20734000000,2748000000,273403000000,None,None,4099000000,None,5656000000,22601000000,4982000000,1938000000,913000000,40189000000,5016000000,350000000,9207000000,2079000000,10476000000,2427000000,69744000000,None,None,0,53688000000,1097000000,151068000000,203659000000,273403000000



 This is 0001652044-19-000032 a 10-Q document.

 Q_final ['Quarter', 0, 'Q3'] Y ['Year', 0, '2019']
Shape -  (1, 44)


,Quarter,Year,Assets,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Income taxes receivable,Inventory,Other current assets,Total current assets,Non marketable investments,Deferred income taxes,Property and Equipment,Operating lease assets,Intangible assets,Goodwill,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q3,2019,None,None,16032000000,105145000000,121177000000,20889000000,192000000,1401000000,4699000000,148358000000,12488000000,564000000,69252000000,10341000000,1747000000,18069000000,2225000000,263044000000,None,None,4142000000,7399000000,21038000000,4835000000,1679000000,131000000,39224000000,4082000000,364000000,11355000000,1747000000,9666000000,1637000000,68075000000,None,None,0,49040000000,1196000000,147125000000,194969000000,263044000000



 This is 0001652044-19-000023 a 10-Q document.

 Q_final ['Quarter', 0, 'Q2'] Y ['Year', 0, '2019']
Shape -  (1, 44)


...

,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Income taxes receivable,Inventory,Other current assets,Total current assets,Non marketable investments,Deferred income taxes,Property and Equipment,Intangible assets,Goodwill,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2017,None,10715000000,91156000000,101871000000,18336000000,369000000,749000000,2983000000,124308000000,7813000000,680000000,42383000000,2692000000,16747000000,2672000000,197295000000,None,None,3137000000,4581000000,10177000000,3975000000,1432000000,881000000,24183000000,3969000000,340000000,12812000000,430000000,3059000000,44793000000,None,None,0,40247000000,992000000,113247000000,152502000000,197295000000



 This is 0001652044-17-000008 a 10-K document.

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2016']
Shape -  (1, 44)


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Receivable under reverse repurchase agreements,Income taxes receivable,Inventory,Prepaid revenue share,Total current assets,Prepaid revenue share,Non marketable investments,Deferred income taxes,Property and Equipment,Intangible assets,Goodwill,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Short term debt,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Securities lending payable,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2016,None,12918000000,73415000000,86333000000,14137000000,0,95000000,268000000,4575000000,105408000000,1819000000,5878000000,383000000,34234000000,3307000000,16468000000,167497000000,None,None,2041000000,0,3976000000,6144000000,2942000000,0,1099000000,554000000,16756000000,3935000000,202000000,4677000000,226000000,2665000000,28461000000,None,None,0,36307000000,2402000000,105131000000,139036000000,167497000000



 This is 0001652044-16-000012 a 10-K document.

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2015']

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2015']
Shape -  (1, 42)


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Receivable under reverse repurchase agreements,Income taxes receivable,Prepaid revenue share,Total current assets,Prepaid revenue share,Non marketable investments,Deferred income taxes,Property and Equipment,Intangible assets,Goodwill,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Short term debt,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Securities lending payable,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Other long term liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive income,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2015,None,16549000000,56517000000,73066000000,11556000000,450000000,1903000000,3139000000,90114000000,3181000000,5183000000,251000000,29016000000,3847000000,15869000000,147461000000,None,None,1931000000,3225000000,3539000000,4768000000,2329000000,2428000000,788000000,302000000,19310000000,1995000000,151000000,3663000000,189000000,1822000000,None,None,0,32982000000,1874000000,89223000000,120331000000,147461000000


In [164]:
QorK

{'10-Q': ['0001652044-20-000021',
  '0001652044-19-000032',
  '0001652044-19-000023',
  '0001652044-19-000015',
  '0001652044-18-000035',
  '0001652044-18-000027',
  '0001652044-18-000016',
  '0001652044-17-000042',
  '0001652044-17-000026',
  '0001652044-17-000014',
  '0001652044-16-000038',
  '0001652044-16-000032',
  '0001652044-16-000021',
  '0001652044-15-000005'],
 '10-K': ['0001652044-20-000008',
  '0001652044-19-000004',
  '0001652044-18-000007',
  '0001652044-17-000008',
  '0001652044-16-000012']}

# Save Files Locally. 

In [162]:
Checklist = {}

for k, v in QorK.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']
        year = int(vv.split(sep='-')[1])
        try:
            # Extract Table from html code and wrap it as a list of df.
            pages = master_filings_dict[vv]['filing_documents'][k]['pages_code']

            df = table(pages, k, year)
            table_df = df[0].drop([1]).reset_index(drop=True)
            
            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
            ICD.display(table_df)
            
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list


 Q_final ['Quarter', 0, 'Q1'] Y ['Year', 0, '2020']
All the pages from 10-Q document 0001652044-20-000021 have been tableized and saved.


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Income taxes receivable,Inventory,Other current assets,Total current assets,Non marketable investments,Deferred income taxes,Property and Equipment,Operating lease assets,Intangible assets,Goodwill,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,NaN,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2020,None,19644000000,97585000000,117229000000,21825000000,1910000000,889000000,5165000000,147018000000,12367000000,730000000,76747000000,11219000000,1840000000,20734000000,2748000000,273403000000,None,None,4099000000,None,5656000000,22601000000,4982000000,1938000000,913000000,40189000000,5016000000,350000000,9207000000,2079000000,10476000000,2427000000,69744000000,None,None,0,53688000000,1097000000,151068000000,203659000000,273403000000


--------------------------------------------------------------------------------

 Q_final ['Quarter', 0, 'Q3'] Y ['Year', 0, '2019']
All the pages from 10-Q document 0001652044-19-000032 have been tableized and saved.


,Quarter,Year,Assets,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Income taxes receivable,Inventory,Other current assets,Total current assets,Non marketable investments,Deferred income taxes,Property and Equipment,Operating lease assets,Intangible assets,Goodwill,Other non current assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q3,2019,None,None,16032000000,105145000000,121177000000,20889000000,192000000,1401000000,4699000000,148358000000,12488000000,564000000,69252000000,10341000000,1747000000,18069000000,2225000000,263044000000,None,None,4142000000,7399000000,21038000000,4835000000,1679000000,131000000,39224000000,4082000000,364000000,11355000000,1747000000,9666000000,1637000000,68075000000,None,None,0,49040000000,1196000000,147125000000,194969000000,263044000000


--------------------------------------------------------------------------------

 Q_final ['Quarter', 0, 'Q2'] Y ['Year', 0, '2019']
All the pages from 10-Q document 0001652044-19-000023 have been tableized and saved.


...

--------------------------------------------------------------------------------

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2016']
All the pages from 10-K document 0001652044-17-000008 have been tableized and saved.


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Receivable under reverse repurchase agreements,Income taxes receivable,Inventory,Prepaid revenue share,Total current assets,Prepaid revenue share,Non marketable investments,Deferred income taxes,Property and Equipment,Intangible assets,Goodwill,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Short term debt,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Securities lending payable,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Other long term liabilities,Total liabilities,Commitments and Contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2016,None,12918000000,73415000000,86333000000,14137000000,0,95000000,268000000,4575000000,105408000000,1819000000,5878000000,383000000,34234000000,3307000000,16468000000,167497000000,None,None,2041000000,0,3976000000,6144000000,2942000000,0,1099000000,554000000,16756000000,3935000000,202000000,4677000000,226000000,2665000000,28461000000,None,None,0,36307000000,2402000000,105131000000,139036000000,167497000000


--------------------------------------------------------------------------------

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2015']

 Q_final ['Quarter', 0, 'Q4'] Y ['Year', 0, '2015']
All the pages from 10-K document 0001652044-16-000012 have been tableized and saved.


,Quarter,Year,Current assets,Cash,Marketable securities,Total cash,Accounts Receivable,Receivable under reverse repurchase agreements,Income taxes receivable,Prepaid revenue share,Total current assets,Prepaid revenue share,Non marketable investments,Deferred income taxes,Property and Equipment,Intangible assets,Goodwill,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Short term debt,Accrued compensation and benefits,Accrued expenses and other current liabilities,Accrued revenue share,Securities lending payable,Deferred Revenue,Income taxes payable,Total current liabilities,Long term debt,Deferred Revenue,Income taxes payable,Deferred income taxes,Other long term liabilities,Commitments and contingencies,Shareholder Equity,Preferred Stock,Common Stock,Accumulated other comprehensive income,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2015,None,16549000000,56517000000,73066000000,11556000000,450000000,1903000000,3139000000,90114000000,3181000000,5183000000,251000000,29016000000,3847000000,15869000000,147461000000,None,None,1931000000,3225000000,3539000000,4768000000,2329000000,2428000000,788000000,302000000,19310000000,1995000000,151000000,3663000000,189000000,1822000000,None,None,0,32982000000,1874000000,89223000000,120331000000,147461000000


--------------------------------------------------------------------------------


In [163]:
Checklist

{'10-Q': [], '10-K': []}

# Comparing the Old and the New.

In [ ]:
# Sample: Older filing style. 
import pickle
pd.set_option('display.max_columns', 100)
with open("0001564590-19-038256-10-Q.csv", "rb") as fp:
    file = pickle.load(fp)
df01 = file[0]

# df01['CSN_SUM'] = df01['Convertible Senior Notes'].values.sum()
# del df01['Convertible Senior Notes']

# df01['DeferredRevenue'] = df01['Deferred Revenue'].values.sum()
# del df01['Deferred Revenue']

# df01['NoncontrollingInterests'] = df01['Noncontrolling interests in subsidiaries'].values.sum()
# del df01['Noncontrolling interests in subsidiaries']

print(df01.shape)
df01

In [ ]:
duplicate_columns( df01 )

### Newer Filing

In [ ]:
# Sample: Newer filing style. 
import pickle 
import pandas as pd
pd.set_option('display.max_columns', 100)
with open("0001564590-20-019931-10-Q.csv", "rb") as fp:
    file = pickle.load(fp)
df02 = file[0]
print(df02.shape) 
df02

In [ ]:
checking_duplicate_columns( df02 )

In [ ]:
print('Using the Function')

duplicate_columns( df02 )


In [ ]:
# Slamming the Raw DataFrame into the dissected column cleanser. 
print('Not Using the Function')
duplicates = [k for (k,v) in Counter(df02.columns).items() if v > 1] 

for dupli in duplicates: 
    res = df02[dupli].values.sum()
    df02['Total {}'.format(dupli)] = res
    df02.drop(dupli, axis=1, inplace=True)
    
df02

In [ ]:
df02.columns

In [ ]:
checking_duplicate_columns( df2 )

# Concating dfs

### Example code: 

In [ ]:
c_ = pd.concat([df2, df01], axis=0, sort=False)

print(c_.shape)
c_

In [ ]:
# Serious Stuff. 
def duplicate_columns( DataF ):
    
    duplicates = [k for (k,v) in Counter(DataF.columns).items() if v > 1] 

    for dupli in duplicates: 
        
        res = DataF[dupli].values.sum()
        
        DataF['Serious {}'.format(dupli)] = res
        
        DataF.drop(dupli, axis=1, inplace=True)

    return DataF

In [ ]:
# Just for checking.
def checking_duplicate_columns( DataF ):
    
    duplicates = [k for (k,v) in Counter(DataF.columns).items() if v > 1] 

    
    print('Shape of df: ', DataF.shape )
    return duplicates